# KPM.jl GPU Test Notebook

Open in Colab: https://colab.research.google.com/github/Pixley-Research-Group-in-CMT/KPM.jl/blob/colab-test/colab_test.ipynb

This notebook automatically posts results to GitHub PR #19.

In [ ]:
# === Cell 1: Install Julia ===
!curl -fsSL https://install.julialang.org | sh -s -- --yes
!pip install julia
import os
os.environ['PATH'] += ':/root/.juliaup/bin'
!julia --version

In [ ]:
# === Cell 2: Clone KPM.jl (main branch) ===
%cd /content
!rm -rf KPM.jl
!git clone https://github.com/Pixley-Research-Group-in-CMT/KPM.jl.git
%cd KPM.jl
!git log --oneline -3

In [ ]:
# === Cell 3: Run KPM tests and save output ===
import subprocess
result = subprocess.run(['julia', '-e', '''
using Pkg
Pkg.add("CUDA")
Pkg.develop(path=".")
using KPM, CUDA
println("=== Extension loaded: ", @isdefined(KPMCUDAExt), " ===")
println("=== CUDA functional: ", CUDA.functional(), " ===")
try
    if CUDA.functional()
        x = rand(10, 10)
        y = KPM.maybe_to_device(x)
        println("maybe_to_device works: ", typeof(y))
    else
        println("No GPU, testing CPU path")
        x = rand(10, 10)
        y = KPM.maybe_to_device(x)
        println("CPU path works: ", typeof(y))
    end
catch e
    println("ERROR: ", e)
    rethrow(e)
end
'''], capture_output=True, text=True)

output = result.stdout + "\n\nSTDERR:\n" + result.stderr
print(output)

# Save to file for next cell
with open('/content/test_output.txt', 'w') as f:
    f.write(output)

In [ ]:
# === Cell 4: Post results to GitHub PR #19 ===
# Enter your GitHub token below (get from https://github.com/settings/tokens)
GITHUB_TOKEN = ""  # <-- paste your token here

import requests
import json
from datetime import datetime

# Read test output
with open('/content/test_output.txt', 'r') as f:
    test_output = f.read()

# Prepare comment
comment_body = f"""
## Colab GPU Test Results

**Timestamp:** {datetime.now().isoformat()}
**Runtime:** Google Colab T4 GPU

### Test Output
```
{test_output}
```

---
*Posted automatically from Colab*
"""

# Post to PR #19
url = "https://api.github.com/repos/Pixley-Research-Group-in-CMT/KPM.jl/issues/19/comments"
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}
data = {"body": comment_body}

response = requests.post(url, headers=headers, json=data)
print(f"Status: {response.status_code}")
print(f"Response: {response.json().get('html_url', response.text)}")

## Instructions

1. **Cell 1-3**: Click Run to execute tests
2. **Cell 4**: Add your GitHub token, then Run
3. Check PR #19 for the posted results